# SITCOMTN-107 - Force Actuator Following Error Movies

As described in [SITCOMTN-107], we are seeing discrepancies between the Force Actuators applied forces and the demanded forces.  
Now, we want to quantify those discrepancies vs time and actuator location.
  

[SITCOMTN-107]: https://sitcomtn-107.lsst.io/
[M1M3 Actuator Forces dashboard in USDF]: https://usdf-rsp.slac.stanford.edu/chronograf/sources/1/dashboards/61?refresh=Paused&tempVars%5Bz_index%5D=112&tempVars%5By_index%5D=0&tempVars%5Bx_index%5D=112&tempVars%5Bs_index%5D=112&lower=now%28%29%20-%205m

## Notebook Preparations

Let's have here all the imports and global variables we will need during the notebook execution.  

In [ ]:
# Directory to store the data
from pathlib import Path
data_dir = Path("./plots")
data_dir.mkdir(exist_ok=True, parents=True)

# "Gentle" slew event
gentle = [20241107, 90]

# "Aggressive" slew event
#aggressive = [20240102, 1308]

In [ ]:
import sys, time, os, asyncio
import shlex, subprocess
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from astropy.time import Time, TimeDelta
from lsst.ts.xml.tables.m1m3 import FATable, FAIndex, force_actuator_from_id, actuator_id_to_index
from lsst_efd_client import EfdClient
from lsst.summit.utils.tmaUtils import TMAEventMaker

## Set up the necessary subroutines

In [ ]:
def heat_map_z(df, axp, axs, FATable, index, zmin, zmax):
    # This builds the following error heat maps
    types = [['SAA','NA', 'o', 'Z'], \
             ['DAA','Y_PLUS', '^', 'Y_PLUS'], \
             ['DAA','Y_MINUS', 'v', 'Y_MINUS'], \
             ['DAA','X_PLUS', '>', 'X_PLUS'], \
             ['DAA','X_MINUS', '<', 'X_MINUS']]
    axp.set_title("Primary")
    axp.set_xlabel("X position (m)")
    axp.set_ylabel("Y position (m)")

    for [type, orient, marker, label] in types:
        xs = []
        ys = []
        zs = []
        for i in range(len(FATable)):
            x = FATable[i].x_position
            y = FATable[i].y_position
            if FATable[i].actuator_type.name == type and \
                FATable[i].orientation.name == orient:
                xs.append(x)
                ys.append(y)
                name=f"primaryCylinderFollowingError{i}"
                zs.append(df.iloc[index][name])
        im = axp.scatter(xs, ys, marker='o', c=zs, cmap='RdBu_r', \
                         vmin=zmin, vmax=zmax, s=50, label=label)
    plt.colorbar(im, ax=axp,fraction=0.055, pad=0.02, cmap='RdBu_r')  
    axs.set_title("Secondary")
    axs.set_xlabel("X position (m)")
    axp.set_xlim(-5,5)
    axp.set_ylim(-5,5)
    axs.set_xlim(-5,5)
    axs.set_ylim(-5,5)

    for [type, orient, marker, label] in types:
        if type == 'SAA':
            continue
        xs = []
        ys = []
        zs = []
        for i in range(len(FATable)):
            x = FATable[i].x_position
            y = FATable[i].y_position
            if FATable[i].actuator_type.name == type and \
                FATable[i].orientation.name == orient:
                xs.append(x)
                ys.append(y)
                name=f"secondaryCylinderFollowingError{FATable[i].s_index}"
                zs.append(df.iloc[index][name])
        im = axs.scatter(xs, ys, marker=marker, c=zs, cmap='RdBu_r', \
                         vmin=zmin, vmax=zmax, s=50, label=label)
    plt.colorbar(im, ax=axs,fraction=0.055, pad=0.02, cmap='RdBu_r')  

def hard_point_plot(df, ax, t, t0, tmin, tmax):
    # This plots the hardpoint forces
    ax.set_title("Hardpoint forces")
    ax.set_ylabel("measuredForce(N)")
    #ax.set_ylim(-3500, 3500)
    ax.set_ylim(-1000, 1000)
    times = df['timestamp'].values - t0
    for i in range(6):
        data = df[f'measuredForce{i}'].values
        ax.plot(times, data)
    ax.set_xlim(tmin, tmax)
    ax.set_xticks([])    
    #ax.plot([t, t], [-3000, 3000], ls='--', color='black')
    #ax.plot([times[0], times[-1]], [3000, 3000], color='red')
    #ax.plot([tmin, tmax], [-3000, -3000], color='red')
    #ax.plot([tmin, tmax], [1000, 1000], ls='--', color='blue')
    ax.plot([tmin, tmax], [500, 500], ls='--', color='blue')
    #ax.plot([tmin, tmax], [-1000, -1000], ls='--', color='blue')
    ax.plot([tmin, tmax], [-500, -500], ls='--', color='blue')

def tma_plot(az, el, ax, t, t0, tmin, tmax):
    #This plots the TMA position
    ax.set_ylabel("TMA Velocity\n(deg/sec)")
    #ax.set_ylim(-10,10)
    ax.set_ylim(-0.5, 0.5)
    ax.set_xlabel("Time (sec)")
    times = az['timestamp'] - t0
    az_v = az['actualVelocity'].values
    el_v = el['actualVelocity'].values
    ax.plot(times, az_v, color='blue', label='Az')
    ax.plot(times, el_v, color='green', label='El')
    ax.set_xlim(tmin, tmax)
    ax.legend()
    ax.plot([t, t], [-3000, 3000], ls='--', color='black')


## Now get the data and generate the frames
### This will take some time

### First, get the needed data

In [ ]:
client = EfdClient('usdf_efd')
[dayObs, seqNum] = gentle
eventMaker = TMAEventMaker()
event = eventMaker.getEvent(dayObs, seqNum)
start = event.begin
end = event.end
dirName = str(data_dir / f"actuator_movie_{dayObs}_{seqNum}")
%mkdir -p {dirName}
pad_start = 1.0 # Starts the plots before the event begin
plot_start = start - TimeDelta(pad_start, format='sec') 
pad_end = 0.0 # continues the plots after the event end
plot_end = end + TimeDelta(pad_end, format='sec') 
forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", \
                                         "*", plot_start, plot_end)
hardpoints = await client.select_time_series("lsst.sal.MTM1M3.hardpointActuatorData", \
                                            "*", plot_start, plot_end)
az = await client.select_time_series('lsst.sal.MTMount.azimuth', \
                                            ['*'],  plot_start, plot_end)
el = await client.select_time_series('lsst.sal.MTMount.elevation', \
                                            ['*'],  plot_start, plot_end) 

t0 = start.unix_tai
# The value below compensates for the different delays in the
# different databases
t0_az_el = 2.0 * start.unix_tai - az['timestamp'].iloc[0] - pad_start
tmax = forces['timestamp'].iloc[-1] - t0
tmin = -pad_start

In [ ]:
len(forces)

### Now, generate the frames.

In [ ]:
# Build the individual frames
zmin = -10.0
zmax = 10.0

fig = plt.figure(figsize=(8,8))
for n in range(0, len(forces), 10):
    t = Time(forces.index[n], scale='utc').unix_tai - t0
    t_msec = int(t * 1000)
    fig.suptitle(f"Actuator following errors." +
                 f"T = {t_msec} msec\n {dayObs} - seqNum {seqNum}", \
                 y=0.90)
    axp = fig.add_axes((0.1, 0.45, 0.35, 0.35))
    axs = fig.add_axes((0.55, 0.45, 0.35, 0.35))
    heat_map_z(forces, axp, axs, FATable, n, zmin, zmax)
    ax_hp = fig.add_axes((0.1, 0.23, 0.8, 0.15))
    hard_point_plot(hardpoints, ax_hp, t, t0, tmin, tmax)
    ax_tma = fig.add_axes((0.1, 0.08, 0.8, 0.15))
    tma_plot(az, el, ax_tma, t, t0_az_el, tmin, tmax)
    plt.savefig(f"{dirName}/Frame_{n:05d}.png")
    plt.clf()
plt.close()

## Now build the movie

In [ ]:
print(f"\033[1mThe movie name will be: " + 
      f"{dirName}/m1m3_movie_{dayObs}_{seqNum}.mp4\033[0m")

command = f"ffmpeg -pattern_type glob -i " + \
    f"'{dirName}/*.png' -f mp4 -vcodec libx264" + \
    f" -pix_fmt yuv420p -framerate 50 -y {dirName}" + \
    f"/m1m3_movie_{dayObs}_{seqNum}.mp4"
args = shlex.split(command)
build_movie = subprocess.Popen(args)
build_movie.wait()